In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
import joblib
from os import walk

import time

%matplotlib inline

In [2]:
#pathBase = 'I://Chuan//Documents//MyData//DICOM//NormalCAG//Arrangement//'
pathBase = 'E://Chuan//Documents//Database//DICOM//CoroBase//Arrangement//'

theWhole = {}

f = []
f_LCA_Caudal = []
mypath_LCA_Caudal = pathBase + 'LCA_Caudal'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Caudal):
    f.extend(filenames)
for l in f:
    f_LCA_Caudal.append(mypath_LCA_Caudal + '\\' + l) 
    
f = []
f_LCA_Cranial = []
mypath_LCA_Cranial = pathBase + 'LCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial.append(mypath_LCA_Cranial + '\\' + l) 
    
f = []
f_LCA_Cranial_LAO = []
mypath_LCA_Cranial_LAO = pathBase + 'LCA_Cranial_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_LAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_LAO.append(mypath_LCA_Cranial_LAO + '\\' + l) 
    
    
f = []
f_LCA_Cranial_RAO = []
mypath_LCA_Cranial_RAO = pathBase + 'LCA_Cranial_RAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_RAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_RAO.append(mypath_LCA_Cranial_RAO + '\\' + l) 
    
f = []
f_LCA_Spider = []
mypath_LCA_Spider = pathBase + 'LCA_Spider'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Spider):
    f.extend(filenames)
for l in f:
    f_LCA_Spider.append(mypath_LCA_Spider + '\\' + l) 
    
f = []
f_RCA_Cranial = []
mypath_RCA_Cranial = pathBase + 'RCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_Cranial):
    f.extend(filenames)
for l in f:
    f_RCA_Cranial.append(mypath_RCA_Cranial + '\\' + l) 
    
f = []
f_RCA_LAO = []
mypath_RCA_LAO = pathBase + 'RCA_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_LAO):
    f.extend(filenames)
for l in f:
    f_RCA_LAO.append(mypath_RCA_LAO + '\\' + l) 
    

theWhole['LCA_Caudal'] = f_LCA_Caudal
theWhole['LCA_Cranial'] = f_LCA_Cranial
theWhole['LCA_Cranial_LAO'] = f_LCA_Cranial_LAO
theWhole['LCA_Cranial_RAO'] = f_LCA_Cranial_RAO
theWhole['LCA_Spider'] = f_LCA_Spider
theWhole['RCA_Cranial'] = f_RCA_Cranial
theWhole['RCA_LAO'] = f_RCA_LAO

In [3]:
theWhole['LCA_Cranial_LAO'][1].split('.')[-1]

'npy'

In [4]:
start = time.time()

array = {}
i = 0

for view in theWhole.keys():
    array[view] = []
    for case in theWhole[view]:
        if case.split('.')[-1] == 'npy':
            arr = np.load(case)
            array[view].append(arr)

end = time.time()
print(end - start)

72.6152777671814


### 2D to 1D

In [5]:
array_flat = {}

for view in array.keys():
    array_flat[view] = []
    for case in array[view]:
        case_flat = []
        for row in case:
            for column in row:
                case_flat.append(column)            
        array_flat[view].append(case_flat)

## Support Vector Classification

In [6]:
x_train_list = []
y_train_list = []

for y, x in array_flat.items():    
    for x_item in x:
        x_train_list.append(x_item)
        y_train_list.append(y)

### Training

In [7]:
start = time.time()

X_train = np.array(x_train_list)
y_train = np.array(y_train_list)

# linear kernel
clf = svm.SVC(kernel='linear', decision_function_shape='ovo')
clf.fit(X_train, y_train)

end = time.time()

In [8]:
print(end - start)

1701.719046831131


### Prediction

In [9]:
print('w = ',clf.coef_)
print('b = ',clf.intercept_)
print('Indices of support vectors = ', clf.support_)
print('Support vectors = ', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)
print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))

w =  [[ 7.33690523e-06  6.29797422e-06  6.43049002e-06 ... -9.48171596e-06
  -8.99306446e-06 -8.60356363e-06]
 [ 1.75547107e-06  1.20906127e-06  1.01077357e-06 ...  9.12109808e-07
   1.17367279e-06  1.87210833e-06]
 [ 1.29571867e-05  1.35839237e-05  1.41636797e-05 ... -3.29884597e-06
  -1.12628206e-06  1.49335199e-06]
 ...
 [-3.60505804e-06 -4.37592565e-06 -4.82044872e-06 ... -4.29642009e-06
  -3.32524540e-06 -2.42620827e-06]
 [-3.77498640e-06 -3.61493338e-06 -3.27436356e-06 ... -5.79782148e-07
  -4.54205081e-07 -4.39920130e-07]
 [-3.82235464e-06 -8.90850032e-07  1.10538434e-06 ... -4.32639197e-06
  -4.83723032e-06 -6.00664784e-06]]
b =  [-1.35879623 -0.247623   -5.86213482 -6.49160635 -0.93991586 -1.37790411
  4.12133406 -0.02014708 -2.67026621  4.18832428 -1.04336236 -1.55026182
 -0.16372721 -0.74734503 -4.36533492 -0.99590247  3.55155726  0.08688072
 -6.91080373 -5.67305271 -1.07768388]
Indices of support vectors =  [  0   2   3   6   8  10  11  15  16  17  19  21  23  25  27  28  2

In [10]:
#pathBase = 'I://Chuan//Documents//MyData//DICOM//NormalCAG//Arrangement//test//'
pathBase = 'E://Chuan//Documents//Database//DICOM//CoroBase//Arrangement//test//'

theWhole = {}

f = []
f_LCA_Caudal = []
mypath_LCA_Caudal = pathBase + 'LCA_Caudal'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Caudal):
    f.extend(filenames)
for l in f:
    f_LCA_Caudal.append(mypath_LCA_Caudal + '\\' + l) 
    
f = []
f_LCA_Cranial = []
mypath_LCA_Cranial = pathBase + 'LCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial.append(mypath_LCA_Cranial + '\\' + l) 
    
f = []
f_LCA_Cranial_LAO = []
mypath_LCA_Cranial_LAO = pathBase + 'LCA_Cranial_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_LAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_LAO.append(mypath_LCA_Cranial_LAO + '\\' + l) 
    
    
f = []
f_LCA_Cranial_RAO = []
mypath_LCA_Cranial_RAO = pathBase + 'LCA_Cranial_RAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_RAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_RAO.append(mypath_LCA_Cranial_RAO + '\\' + l) 
    
f = []
f_LCA_Spider = []
mypath_LCA_Spider = pathBase + 'LCA_Spider'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Spider):
    f.extend(filenames)
for l in f:
    f_LCA_Spider.append(mypath_LCA_Spider + '\\' + l) 
    
f = []
f_RCA_Cranial = []
mypath_RCA_Cranial = pathBase + 'RCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_Cranial):
    f.extend(filenames)
for l in f:
    f_RCA_Cranial.append(mypath_RCA_Cranial + '\\' + l) 
    
f = []
f_RCA_LAO = []
mypath_RCA_LAO = pathBase + 'RCA_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_LAO):
    f.extend(filenames)
for l in f:
    f_RCA_LAO.append(mypath_RCA_LAO + '\\' + l) 
    

theWhole['LCA_Caudal'] = f_LCA_Caudal
theWhole['LCA_Cranial'] = f_LCA_Cranial
theWhole['LCA_Cranial_LAO'] = f_LCA_Cranial_LAO
theWhole['LCA_Cranial_RAO'] = f_LCA_Cranial_RAO
theWhole['LCA_Spider'] = f_LCA_Spider
theWhole['RCA_Cranial'] = f_RCA_Cranial
theWhole['RCA_LAO'] = f_RCA_LAO

In [11]:
start = time.time()

array = {}
i = 0

for view in theWhole.keys():
    array[view] = []
    for case in theWhole[view]:
        if case.split('.')[-1] == 'npy':
            arr = np.load(case)
            array[view].append(arr)

end = time.time()
print(end - start)

71.82448625564575


### 2D to 1D

In [12]:
array_flat = {}

for view in array.keys():
    array_flat[view] = []
    for case in array[view]:
        case_flat = []
        for row in case:
            for column in row:
                case_flat.append(column)            
        array_flat[view].append(case_flat)

In [13]:
x_test_list = []
y_test_list = []

for y, x in array_flat.items():    
    for x_item in x:
        x_test_list.append(x_item)
        y_test_list.append(y)

In [14]:
X_test = np.array(x_test_list)
y_test = np.array(y_test_list)

## Performance Measures

In [15]:
y_predict = clf.predict(X_test)
y_test = y_test

In [16]:
y_predict

array(['LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'RCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial_RAO', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial_RAO', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial_LAO', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', 'LCA_Cranial', 'LCA_Cranial', 'RCA_Cranial',
       'LCA_Cranial_RAO', 'LCA_Cranial_RAO', 'LCA_Cranial_RAO',
       'LCA_Cranial_RAO', 'LCA_Cranial', 'LCA_Cranial_RAO',
       'LCA_Cranial_RAO', 'LCA_Cranial_RAO', 'LCA_Cranial_RAO',
       'LCA_Cranial_RAO', 'LCA_Cranial_RAO', 'LCA_Spider', 'RCA_Cranial',
       'LCA_Spider', 'LCA_Spider', 'LCA_Spider', 'LCA_Spider',
       'LCA_Spider', 'LCA_Spider', 'LCA_Spider', 'LCA_Spider',
       'RCA_C

In [17]:
y_test

array(['LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial_LAO', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', 'LCA_Cranial_RAO', 'LCA_Cranial_RAO',
       'LCA_Cranial_RAO', 'LCA_Cranial_RAO', 'LCA_Cranial_RAO',
       'LCA_Cranial_RAO', 'LCA_Cranial_RAO', 'LCA_Cranial_RAO',
       'LCA_Cranial_RAO', 'LCA_Cranial_RAO', 'LCA_Cranial_RAO',
       'LCA_Spider', 'LCA_Spider', 'LCA_Spider', 'LCA_Spider',
       'LCA_Spider', 'LCA_Spider', 'LCA_Spider', 'LCA_Spider',
       'LCA_Spider', 'LCA_Spider',

### Confusion Matrix (混淆矩阵)

In [18]:
cm = np.array(confusion_matrix(y_test, y_predict, labels=['LCA_Caudal', 'LCA_Cranial', 
                                                          'LCA_Cranial_LAO', 'LCA_Cranial_RAO', 
                                                          'LCA_Spider', 
                                                          'RCA_Cranial', 'RCA_LAO']))
confusion = pd.DataFrame(cm, index=['LCA_Caudal', 'LCA_Cranial', 
                                    'LCA_Cranial_LAO', 'LCA_Cranial_RAO', 'LCA_Spider',
                                    'RCA_Cranial', 'RCA_LAO'],
                        columns=['Predictive LCA_Caudal', 'Predictive LCA_Cranial',
                                 'Predictive LCA_Cranial_LAO', 'Predictive LCA_Cranial_RAO',
                                 'Predictive LCA_Spider',
                                    'Predictive RCA_Cranial', 'Predictive RCA_LAO'])
confusion.to_csv('ConfusionMatrix.csv')
confusion

,Predictive LCA_Caudal,Predictive LCA_Cranial,Predictive LCA_Cranial_LAO,Predictive LCA_Cranial_RAO,Predictive LCA_Spider,Predictive RCA_Cranial,Predictive RCA_LAO
LCA_Caudal,10,0,0,0,0,0,0
LCA_Cranial,0,11,0,2,0,1,0
LCA_Cranial_LAO,0,2,7,0,0,1,0
LCA_Cranial_RAO,0,1,0,10,0,0,0
LCA_Spider,0,0,0,0,9,1,0
RCA_Cranial,0,0,1,0,0,7,2
RCA_LAO,0,0,0,0,0,0,11


In [19]:
print(classification_report(y_test, y_predict))

                 precision    recall  f1-score   support

     LCA_Caudal       1.00      1.00      1.00        10
    LCA_Cranial       0.79      0.79      0.79        14
LCA_Cranial_LAO       0.88      0.70      0.78        10
LCA_Cranial_RAO       0.83      0.91      0.87        11
     LCA_Spider       1.00      0.90      0.95        10
    RCA_Cranial       0.70      0.70      0.70        10
        RCA_LAO       0.85      1.00      0.92        11

       accuracy                           0.86        76
      macro avg       0.86      0.86      0.86        76
   weighted avg       0.86      0.86      0.85        76



## Save the Model

In [21]:
c = joblib.dump(clf, 'trained_model.m')
print(c)

['trained_model.m']


## Load the Model

In [22]:
d = joblib.load('trained_model.m')
print(d)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
